In [3]:
import whisper
import clip
import torch
import faiss
import os
from PIL import Image
from langchain.vectorstores import FAISS as LangFAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.schema import Document
from langchain_huggingface import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter

device = "cuda" if torch.cuda.is_available() else "cpu"

# 1. Whisper (audio to text)
def transcribe_audio(file_path):
    print("🎤 Transcribing audio with Whisper...")
    model = whisper.load_model("base")
    result = model.transcribe(file_path)
    return result["text"]

# 2. CLIP (image to embedding)
def embed_image(file_path, model, preprocess):
    print("🖼️ Embedding image using CLIP...")
    image = preprocess(Image.open(file_path)).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image)
    return image_features.cpu().numpy()

# 3. Text embedding
def embed_text_chunks(docs):
    print("📚 Embedding text chunks...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    vectorstore = LangFAISS.from_documents(docs, embeddings)
    return vectorstore

# 4. Split large text corpus
def split_documents(texts):
    docs = [Document(page_content=t) for t in texts]
    splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    return splitter.split_documents(docs)

# 5. Load LLM (Light model for Colab)
def load_llm():
    print("🧠 Loading FLAN-T5-Base...")
    model_id = "google/flan-t5-base"
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id)
    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256)
    return HuggingFacePipeline(pipeline=pipe)

# 6. Load CLIP
def init_clip():
    print("⚙️ Loading CLIP model...")
    model, preprocess = clip.load("ViT-B/32", device=device)
    return model, preprocess

# 7. Create QA chain
def create_multimodal_qa(corpus):
    chunks = split_documents(corpus)
    vectorstore = embed_text_chunks(chunks)
    llm = load_llm()
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    return RetrievalQA.from_chain_type(llm=llm, retriever=retriever, return_source_documents=True)

# 8. Chat interface
def start_multimodal_chat():
    clip_model, clip_preprocess = init_clip()

    corpus = [
        "The sun is a star at the center of the Solar System.",
        "Cats are small, carnivorous mammals that are often kept as pets.",
        "The Eiffel Tower is a famous landmark in Paris, France."
    ]

    qa_chain = create_multimodal_qa(corpus)
    clip_text_model = SentenceTransformer("clip-ViT-B-32")
    corpus_embeddings = clip_text_model.encode(corpus)
    index = faiss.IndexFlatL2(corpus_embeddings.shape[1])
    index.add(corpus_embeddings)

    print("🧠 Multimedia RAG Ready!\nType 'exit' to quit.\n")

    while True:
        mode = input("Choose input mode (text/image/audio): ").strip().lower()
        if mode == "exit":
            print("👋 Exiting.")
            break
        elif mode == "text":
            query = input("📝 Your question: ")
        elif mode == "image":
            path = input("📁 Enter image path: ")
            image_features = embed_image(path, clip_model, clip_preprocess)
            D, I = index.search(image_features, k=1)
            query = corpus[I[0][0]]
            print(f"🧠 Interpreted image as: {query}")
        elif mode == "audio":
            path = input("🎵 Enter audio file path: ")
            query = transcribe_audio(path)
            print(f"🧠 Transcribed audio: {query}")
        else:
            print("❌ Invalid mode.")
            continue

        try:
            result = qa_chain.invoke({"query": query})
            print(f"\n🤖 Answer: {result['result']}\n")
        except Exception as e:
            print(f"⚠️ Error: {e}")

# Run
if __name__ == "__main__":
    start_multimodal_chat()


⚙️ Loading CLIP model...
📚 Embedding text chunks...


<ipython-input-3-58ca0b5215e2>:36: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

🧠 Loading FLAN-T5-Base...


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


...

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/961k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


🧠 Multimedia RAG Ready!
Type 'exit' to quit.

Choose input mode (text/image/audio): text
📝 Your question: MS DHONT

🤖 Answer: Cats

Choose input mode (text/image/audio): image
📁 Enter image path: /content/Screenshot 2025-04-02 221515.png
🖼️ Embedding image using CLIP...
🧠 Interpreted image as: The Eiffel Tower is a famous landmark in Paris, France.

🤖 Answer: Cats are small, carnivorous mammals that are often kept as pets.

Choose input mode (text/image/audio): exit
👋 Exiting.
